In [1]:
import pandas as pd
train_2022 = pd.read_csv('train_2022.csv')
data_augmentation_chatGPT = pd.read_csv('data_augmentation_chatGPT.csv')
data_augmentation_random_2_words = pd.read_csv('data_augmentation_random_2_words.csv')
data_augmentation_random_3_words = pd.read_csv('data_augmentation_random_3_words.csv')
translated_en_data = pd.read_csv('translated_en_data.csv')

In [6]:
merged_data = pd.concat([train_2022, data_augmentation_chatGPT], ignore_index=True)
merged_data

,row_id,TEXT,LABEL
0,0,director dirk shafer and co-writer greg hinton...,0
1,1,"a charming , quirky and leisurely paced scotti...",1
2,2,"the price was good , and came quickly though ...",1
3,3,i was looking forward to this game for a coupl...,0
4,4,arguably the year 's silliest and most incoher...,0
...,...,...,...
3995,1995,A creative comedy/thriller.,1
3996,1996,Explores paranoia and insecurity in America's ...,1
3997,1997,Good for power grating.,1
3998,1998,McGrath's variation on the novel crafts moving...,1


In [7]:
final_train_data = merged_data.copy()
final_train_data['LABEL'] = final_train_data['LABEL'].replace(1, 2)
final_train_data

,row_id,TEXT,LABEL
0,0,director dirk shafer and co-writer greg hinton...,0
1,1,"a charming , quirky and leisurely paced scotti...",2
2,2,"the price was good , and came quickly though ...",2
3,3,i was looking forward to this game for a coupl...,0
4,4,arguably the year 's silliest and most incoher...,0
...,...,...,...
3995,1995,A creative comedy/thriller.,2
3996,1996,Explores paranoia and insecurity in America's ...,2
3997,1997,Good for power grating.,2
3998,1998,McGrath's variation on the novel crafts moving...,2


In [5]:
%%time

from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence

# 检查 CUDA 是否可用并设置设备
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"
print(f"Using device: {device}")

# 加载预训练的 RoBERTa tokenizer 和模型
tokenizer = RobertaTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')
model = RobertaForSequenceClassification.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment')

# 将模型转移到指定的设备
model = model.to(device)

# 假设 raw_data 是你的 DataFrame 并已经准备好
train_data = final_train_data.copy()

# 使用 tokenizer 将文本转换为 token IDs
def tokenize_text(text):
    return tokenizer(text, return_tensors='pt', padding=True, truncation=True)['input_ids'].squeeze()

train_data['input_ids'] = train_data['TEXT'].apply(tokenize_text)

# 转换成可以输入模型的格式
inputs = pad_sequence(train_data['input_ids'].tolist(), batch_first=True).to(device)

# 将数据拆分为训练集和测试集
train_inputs, test_inputs, train_labels, test_labels = train_test_split(inputs, train_data['LABEL'].tolist(), test_size=0.2)

# 将标签也转移到 CUDA
train_labels = torch.tensor(train_labels).to(device)
test_labels = torch.tensor(test_labels).to(device)

# 创建 PyTorch DataLoader
train_dataset = TensorDataset(train_inputs, train_labels)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# 定义 optimizer 和损失函数
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# 训练模型
model.train()
for epoch in range(5):
    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs.logits, labels)
        loss.backward()
        optimizer.step()

# 评估模型
model.eval()
with torch.no_grad():
    outputs = model(test_inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# 将预测结果从 CUDA 转移到 CPU 并转换为 NumPy 数组
predictions = predictions.cpu().numpy()

# 轉換格式
modified_predictions = np.where(predictions == 2, 1, predictions)
modified_test_labels = np.where(test_labels.cpu() == 2, 1, test_labels.cpu())
# 打印预测结果
report = classification_report(modified_test_labels, modified_predictions)  # 確保標籤也在 CPU 上
print(report)

D:\Anaconda3\envs\cuda\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\transformers\optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 930.00 MiB. GPU 

In [34]:
import datetime
import pytz
import pandas as pd
def export_csv(df,name):
  now = datetime.datetime.now().astimezone(pytz.timezone('Asia/Taipei'))
  formatted_time = now.strftime('%Y%m%d')
  df.to_csv('result/'+ formatted_time + '_' + name + ".csv", index=False,encoding="utf_8_sig")

In [35]:
%%time
import pandas as pd
import torch
from transformers import RobertaTokenizer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, TensorDataset

# 加载测试数据
test_data = pd.read_csv('test_no_answer_2022.csv')
# 使用与训练数据相同的Tokenizer处理文本
def tokenize_text(text):
    return tokenizer(text, return_tensors='pt', padding='max_length', max_length=512, truncation=True)['input_ids'].squeeze()

test_data['input_ids'] = test_data['TEXT'].apply(tokenize_text)

# 将处理后的文本转换为模型可以接收的格式
test_inputs = pad_sequence(test_data['input_ids'].tolist(), batch_first=True).to(device)

# 创建 DataLoader
test_dataset = TensorDataset(test_inputs)
test_loader = DataLoader(test_dataset, batch_size=8)  # 可调整batch size

# 使用模型进行预测
model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        inputs = batch[0]
        outputs = model(inputs)
        pred = torch.argmax(outputs.logits, dim=-1)
        predictions.extend(pred.cpu().numpy())

# 转换格式，将类别2映射为1，如果需要
predictions = np.where(np.array(predictions) == 2, 1, np.array(predictions))

# 显示或保存预测结果
print(predictions)

# 可选择将预测结果保存到CSV文件
results = pd.DataFrame({
    "row_id": test_data['row_id'],
    "prediction": predictions
})
export_csv(results,'twitter')

[1 1 0 ... 0 1 0]
CPU times: total: 14h 20min 50s
Wall time: 4h 29min 15s
